In [1]:
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'apt'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


댓글 추출 추가한 버전


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

def extract_song_and_comments(year):
    global song_df

    try:
        age_restrictions = driver.find_elements(By.CSS_SELECTOR, "span[title='19세 미만 청소년 이용불가']")

        if age_restrictions:
                print("19금 페이지, 넘어감")
                return  # 현재 함수를 종료하고 다음 노래로 넘어감
        
        # 각 항목 추출 (여기에 예외 처리 추가)
        try:
            song_name = driver.find_element(By.CSS_SELECTOR, "div.song_name").text
        except :
            song_name = None

        try:
            artist_name = driver.find_element(By.CSS_SELECTOR, "a.artist_name span").text
        except :
            artist_name = None
        
        try:
            album_name = driver.find_element(By.CSS_SELECTOR, "dl.list dd:nth-child(2) a").text
        except :
            album_name = None

        try:
            album_to_click = driver.find_element(By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(2) > a")
            album_to_click.click()
            time.sleep(2)
            album_more_to_click = driver.find_element(By.CSS_SELECTOR, "#conts > div.section_albuminfo > div.cont_albuminfo > button")
            album_more_to_click.click()   
            album_story = driver.find_element(By.CSS_SELECTOR, "#d_video_summary > div").text
        except :
            album_story = None
            
        #genre = driver.find_element(By.CSS_SELECTOR, "dl.list dd:nth-child(6)").text
        #release_date = driver.find_element(By.CSS_SELECTOR, "dl.list dd:nth-child(4)").text #downloadfrm > div > div > div.entry > div.meta > dl > dt:nth-child(3)
        #lyrics = driver.find_element(By.CSS_SELECTOR, "div.lyric#d_video_summary").text  # 가사 추가
        
        # 댓글 추출
        #driver.execute_script("document.querySelector('a[href=\"javascript:goReview();\"]').click();")
        #time.sleep(1)
        #driver.execute_script("document.querySelector('a[title=\"추천순으로 보기\"]').click();")
        #time.sleep(1)
        #comments = driver.execute_script("""
        #const elements = document.querySelectorAll("div.list_cmt div.cmt_text.d_cmtpgn_cmt_full_contents");
        #return Array.from(elements).map(element => {
        #  return Array.from(element.childNodes)
        #    .filter(node => node.nodeType === 3)
        #    .map(textNode => textNode.textContent.trim())
        #    .join(" ");
        #});
        #""")

        #comments = " , ".join(comments[:])

        driver.back()

        # 노래 정보와 댓글을 DataFrame에 추가
        new_row = pd.DataFrame({
            '년도': [year],
            '가수명': [artist_name],
            '노래제목': [song_name],
            '앨범명' : [album_name],
            #'장르': [genre],
            #'발매일': [release_date],
            #'가사': [lyrics],
            #'댓글': [comments],
            '앨범소개' : [album_story]
        })
        global song_df
        song_df = pd.concat([song_df, new_row], ignore_index=True)
    except Exception as e:
        print(f"Error: {e}")
        
        print(f"{artist_name:<10} {song_name}\n")

        song_df = pd.concat([song_df, new_row], ignore_index=True)

#conts > div.section_albuminfo > div.cont_albuminfo > button > span



# 크롬 드라이버 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 노래 데이터를 저장할 DataFrame 설정
song_df = pd.DataFrame(columns=['년도', '가수명', '노래제목', '앨범명', '앨범소개'])

# 각 년도별로 반복
for year in range(2010, 2022):
    # 멜론 차트 페이지로 이동
    url = f"https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={year}"
    driver.get(url)
    time.sleep(2)  # 페이지 로드 될때까지 대기, 
    
    # 상위 1-50위 곡 정보 추출
    num_of_songs_50 = driver.execute_script("return document.querySelectorAll('tr.lst50 a.btn.btn_icon_detail').length;")
    for idx in range(num_of_songs_50):
        print(f'in {year}, no.{idx}')
        driver.execute_script(f"document.querySelectorAll('tr.lst50 a.btn.btn_icon_detail')[{idx}].click();")
        time.sleep(2)
        #album_to_click = driver.find_element(By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(2) > a")
        #album_to_click.click()
        extract_song_and_comments(year)
        driver.back()
        time.sleep(2)

    # 차트 페이지에서 51-100위 곡으로 이동
    driver.execute_script("document.querySelector('a[title=\"51 - 100 위 - 페이지 이동\"]').click();")
    time.sleep(2)

    # 상위 51-100위 곡 정보 추출
    num_of_songs_100 = driver.execute_script("return document.querySelectorAll('tr.lst100 a.btn.btn_icon_detail').length;")
    for idx in range(num_of_songs_100):
        print(f'in {year}, no.{idx+50}')
        driver.execute_script(f"document.querySelectorAll('tr.lst100 a.btn.btn_icon_detail')[{idx}].click();")
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, "a[href*='melon.link.goAlbumDetail']")
        extract_song_and_comments(year)
        driver.back()
        time.sleep(2)

    #테스트 출력
    print(song_df)
    # 데이터를 CSV 파일로 저장
    song_df.to_csv(f'~/Downloads/melon_{year}_top100.csv', index=False, encoding='utf-8-sig')
    # 데이터 프레임 비우기
    song_df = pd.DataFrame(columns=song_df.columns)


# 크롬 드라이버 종료
driver.quit()

song_df.head(5)

in 2010, no.0
미쓰에이       Bad Girl Good Girl

in 2010, no.1
아이유        잔소리 (With 2AM 슬옹)

in 2010, no.2
MC몽        죽을 만큼 아파서 (Feat. 멜로우)

in 2010, no.3
포맨 (4MEN)  못해 (Feat. 美)

in 2010, no.4


KeyboardInterrupt: 

In [8]:
song_df

,년도,가수명,노래제목,앨범명,앨범소개
0,2010,미쓰에이,Bad Girl Good Girl,Bad But Good,"A Class실력파 그룹 miss A, 데뷔앨범 ‘Bad but Good’공개.\n..."
1,2010,아이유,잔소리 (With 2AM 슬옹),잔소리,소녀디바 아이유와 2AM 슬옹이 함께 선사하는 달콤 쌉싸래한 LOVE SONG [잔...
2,2010,MC몽,죽을 만큼 아파서 (Feat. 멜로우),블루브랜드 Trauma Part 2,"MC몽-슈프림팀 등 더욱 화려해진 라인업! 대한민국 최고의 프로듀서 김건우,\n힙합..."
3,2010,포맨 (4MEN),못해 (Feat. 美),The 3rd Generation (Special Album),"국내에 알앤비라는 장르가 흥행한 이후, 호소력 짙은 보컬 톤과 화려한 애드립을 구사..."
